In [1]:
import pandas as pd
import numpy as np
!pip install protobuf==4.25.3
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import tqdm
import nltk
import os

from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout, add
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.translate.bleu_score import corpus_bleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.0 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatib

2025-11-21 17:22:12.588221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763745732.952688      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763745733.042264      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
captions_path = "/kaggle/input/flickr8k/captions.txt"
images_path = "/kaggle/input/flickr8k/Images"

In [3]:
num_images = len(os.listdir(images_path))
print("Total images:", num_images)

num_captions = sum(1 for _ in open(captions_path, 'r')) - 1 # -1 for the header line
print("Total captions:", num_captions)

Total images: 8091
Total captions: 40455


In [4]:
def load_captions(filepath):
    captions = {}
    with open(filepath, 'r') as f:
        next(f, None)  # skip header if it exists
        for line in f:
            
            if ',' in line:
                parts = line.split(',', 1)

            image_id, caption = parts[0], parts[1].strip()

            if image_id not in captions:
                captions[image_id] = []

            captions[image_id].append(caption)
    return captions


captions = load_captions(captions_path)
print("Unique image IDs:", len(captions))


Unique image IDs: 8091


In [5]:
for i, (img_id, caption_list) in enumerate(captions.items()):
    for caption in caption_list:
        print(img_id, ":", caption)
    if i == 2:   # stop after printing 3 image IDs (0, 1, 2)
        break


1000268201_693b08cb0e.jpg : A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg : A girl going into a wooden building .
1000268201_693b08cb0e.jpg : A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg : A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg : A little girl in a pink dress going into a wooden cabin .
1001773457_577c3a7d70.jpg : A black dog and a spotted dog are fighting
1001773457_577c3a7d70.jpg : A black dog and a tri-colored dog playing with each other on the road .
1001773457_577c3a7d70.jpg : A black dog and a white dog with brown spots are staring at each other in the street .
1001773457_577c3a7d70.jpg : Two dogs of different breeds looking at each other on the road .
1001773457_577c3a7d70.jpg : Two dogs on pavement moving toward each other .
1002674143_1b742ab4b8.jpg : A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .
1002674143_1b

In [6]:
import string

def clean_captions(captions_dict):
    table = str.maketrans('', '', string.punctuation)

    # maketrans(x, y, z) means:
    # x → characters to replace
    # y → characters to replace them with
    # z → characters to delete
    
    cleaned = {}
    for img_id, caption_list in captions_dict.items():
        cleaned[img_id] = []
        for caption in caption_list:
            caption = caption.lower().translate(table)
            caption = caption.strip()
            caption = ' '.join([word for word in caption.split() if len(word) > 1])
            caption = 'startseq ' + caption + ' endseq'
            cleaned[img_id].append(caption)
    return cleaned

captions_cleaned = clean_captions(captions)
print("Example:", captions_cleaned[list(captions_cleaned.keys())[0]][0])

Example: startseq child in pink dress is climbing up set of stairs in an entry way endseq


In [7]:
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

model_xcep = Xception(include_top=False, pooling='avg')
features = {}

for img_id in tqdm.tqdm(captions_cleaned.keys()):
    img_path = os.path.join(images_path, img_id)
    img = load_img(img_path, target_size=(299, 299))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    feature = model_xcep.predict(img, verbose=0)
    features[img_id] = feature[0]

print("Extracted features:", len(features))

2025-11-21 17:22:36.619787: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


100%|██████████| 8091/8091 [39:35<00:00,  3.41it/s]

Extracted features: 8091


# ⚠️ Save this as a .pkl file once done so you don’t re-run it again (it’s slow)

In [8]:
import pickle
pickle.dump(features, open("/kaggle/working/xception_features.pkl", "wb"))

In [9]:
import pickle
features = pickle.load(open("/kaggle/working/xception_features.pkl", "rb"))

In [10]:
all_captions = []
for key in captions_cleaned:
    all_captions.extend(captions_cleaned[key])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1

max_length = max(len(caption.split()) for caption in all_captions)
print("Vocab size:", vocab_size)
print("Max caption length:", max_length)

Vocab size: 8811
Max caption length: 34


In [11]:
pickle.dump(tokenizer, open("/kaggle/working/tokenizer.pkl", "wb"))

In [12]:
tokenizer = pickle.load(open("tokenizer.pkl", "rb"))

In [13]:
def create_sequences(tokenizer, max_length, captions_list, image_feature, vocab_size):
    X1, X2, y = [], [], []
    for caption in captions_list:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(image_feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [14]:
# feature extractor (encoder)
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# sequence processor (decoder)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (merge both)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 34)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 34, 256)   │  2,255,616 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 34, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 34)        │          0 │ input_layer_2[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8811)      │  2,264,427 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,635,691 (21.50 MB)

 Trainable params: 5,635,691 (21.50 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
sample_image_ids = list(captions_cleaned.keys())[:250]  # first 250 images
X1, X2, y = [], [], []

for img_id in sample_image_ids:
    # flatten feature (1, 2048) → (2048,)
    feature_vec = features[img_id]
    if feature_vec.ndim == 2:   # only if shape is (1, 2048)
        feature_vec = feature_vec.reshape(2048,)

    x1, x2, y_ = create_sequences(
        tokenizer, max_length, captions_cleaned[img_id], feature_vec, vocab_size
    )
    X1.append(x1)
    X2.append(x2)
    y.append(y_)

X1 = np.vstack(X1)
X2 = np.vstack(X2)
y = np.vstack(y)

model.fit([X1, X2], y, epochs=20, batch_size=64, verbose=2)


Epoch 1/20
203/203 - 44s - 215ms/step - loss: 5.8250
Epoch 2/20
203/203 - 40s - 195ms/step - loss: 4.6730
Epoch 3/20
203/203 - 38s - 188ms/step - loss: 3.9885
Epoch 4/20
203/203 - 39s - 191ms/step - loss: 3.4355
Epoch 5/20
203/203 - 40s - 197ms/step - loss: 2.9846
Epoch 6/20
203/203 - 40s - 199ms/step - loss: 2.5951
Epoch 7/20
203/203 - 40s - 198ms/step - loss: 2.2885
Epoch 8/20
203/203 - 43s - 210ms/step - loss: 2.0399
Epoch 9/20
203/203 - 41s - 203ms/step - loss: 1.8422
Epoch 10/20
203/203 - 43s - 211ms/step - loss: 1.6802
Epoch 11/20
203/203 - 42s - 209ms/step - loss: 1.5356
Epoch 12/20
203/203 - 41s - 200ms/step - loss: 1.4003
Epoch 13/20
203/203 - 39s - 190ms/step - loss: 1.3034
Epoch 14/20
203/203 - 42s - 208ms/step - loss: 1.1927
Epoch 15/20
203/203 - 40s - 195ms/step - loss: 1.1007
Epoch 16/20
203/203 - 38s - 189ms/step - loss: 1.0124
Epoch 17/20
203/203 - 38s - 189ms/step - loss: 0.9314
Epoch 18/20
203/203 - 39s - 190ms/step - loss: 0.8661
Epoch 19/20
203/203 - 41s - 203ms/ste

In [17]:
from nltk.translate.bleu_score import corpus_bleu

# generate caption (inference function)
def generate_caption(model, tokenizer, photo, max_length):

    photo = np.expand_dims(photo, axis=0)
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = np.argmax(model.predict([photo, sequence], verbose=0))
        word = next((w for w, i in tokenizer.word_index.items() if i == yhat), None)
        if word is None: break
        in_text += ' ' + word
        if word == 'endseq': break
    return in_text

# example
img_id = list(features.keys())[0]
photo = features[img_id]
caption = generate_caption(model, tokenizer, photo, max_length)
print("Generated caption:", caption)


Generated caption: startseq little girl climbing into wooden building endseq
